**UNIVERSITY OF VICTORIA**\
**CSC 502 PROJECT (202001)**\
**SUBMITTED BY**\
Suseela Pattamatta


**NOTE TO MARKER**\
Instead of connecting to google drive, I have chosen to upload the files. When running the cells, it will prompt for uploading for files. 

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [0]:
!ls

sample_data  spark-2.4.4-bin-hadoop2.7	spark-2.4.4-bin-hadoop2.7.tgz


In [0]:
!rm -rf input_docs*
!rm -rf input_docs_sample.zip


from google.colab import files
uploaded = files.upload()

Saving input_docs.zip to input_docs.zip


In [0]:
!unzip input_docs.zip
!ls

Streaming output truncated to the last 5000 lines.
  inflating: input_docs/5064.html    
  inflating: input_docs/5065.html    
  inflating: input_docs/5066.html    
  inflating: input_docs/5067.html    
  inflating: input_docs/5069.html    
  inflating: input_docs/507.html     
  inflating: input_docs/5070.html    
  inflating: input_docs/5071.html    
  inflating: input_docs/5072.html    
  inflating: input_docs/5073.html    
  inflating: input_docs/5074.html    
  inflating: input_docs/5075.html    
  inflating: input_docs/5076.html    
  inflating: input_docs/5077.html    
  inflating: input_docs/5078.html    
  inflating: input_docs/508.html     
  inflating: input_docs/5080.html    
  inflating: input_docs/5081.html    
  inflating: input_docs/5082.html    
  inflating: input_docs/5083.html    
  inflating: input_docs/5084.html    
  inflating: input_docs/5085.html    
  inflating: input_docs/5086.html    
  inflating: input_docs/5087.html    
  inflating: input_docs/5088.html    

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Create an RDD from a text file**

Each line of the text file becomes an element of the RDD.

In [0]:
# wholeTextFiles generates an RDD of pair values, 
# where the key is the full path of each file, the value is the content of each file
#input = sc.wholeTextFiles("/content/drive/My\ Drive/input_docs");
input = sc.wholeTextFiles("input_docs");

# Now we strip the prefix of filenames and leave only the basename. 
# e.g. 'file:/content/drive/My Drive/Colab Notebooks/data_spark/input_docs/3.html'
# becomes '3.html' 
import os

#(did,text)
input2 = input.map(lambda x: (int(os.path.basename(x[0]).split(".")[0]), x[1]))

print(input2.take(1))

[(13972, "<H2> 7-APR-1987 10:49:16.10</H2>\r\n<H2>IA SO-MINN DIRECT HOGS UP 0.50/1.00 DLR - USDA</H2>\r\nbarrow and gilt prices were mostly\n0.50 dlr higher, instances up 1.00 dlr than monday's\nmidsession, the usda said. Top 52.00 dlrs per cwt.\n    us 1-3 210-250 lbs 49.50-50.50, few 49.00-51.00 at country\npoints.\n    us 1-3 210-250 lbs 50.00-51.50, few 52.00 at packing\nplants.\n    sows - firm/up 1.00 dlr. top 45.00 dlrs.\n    us 1-3 300-650 lbs 40.00-44.00 at country.\n    us 1-3 300-650 lbs 41.00-45.00 at plants.\n \n ")]


In [0]:
# Doc to wordlist function
# The output will be a list of tuples such as 
# ("apple", (3,10,10/20)), 
# where 3 is docid, 
# 10 is frequency of "apple" in this doc, 
# 20 is maxf in in this doc.

from bs4 import BeautifulSoup
from collections import Counter
import re

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# for a given doc return a list of tuples of the form (w, (docid, freq, freq/maxfreq))
def dw(docid, htmltext):
  
  cleantext = BeautifulSoup(htmltext).get_text()
  
  #terms = cleantext.split()
  terms = re.findall('[a-zA-Z]+', cleantext)
  
  # CHANGE ALL TERMS TO LOWERCASE
  terms = [term.lower() for term in terms]
  
  # REMOVE STOPWORDS
  terms = [term for term in terms if term not in stop_words]
  
  maxf = 0
  dict_terms = {}
  for term in terms:
    if term not in dict_terms:
      dict_terms[term] = 1
    else:
      dict_terms[term] += 1

    if dict_terms[term] > maxf:
      maxf = dict_terms[term]

  final_list = []

  for key, value in dict_terms.items():
    final_list.append((key, (docid, value, value/maxf)))

  return final_list   
  
word_docid_freq_tf = input2.flatMap(lambda x: dw(x[0],x[1]))

print(word_docid_freq_tf.take(2))


[('apr', (13972, 1, 0.25)), ('ia', (13972, 1, 0.25))]


In [0]:
# Now create an RDD as follows 
# (word, [(did1,freq1,tf1), (did2,freq2,tf2), ...])
		    
word_postinglist_freq_tf = word_docid_freq_tf.groupByKey()

In [0]:
print(word_postinglist_freq_tf.map(lambda x : (x[0], list(x[1]))).take(4))

[('apr', [(13972, 1, 0.25), (17227, 1, 0.16666666666666666), (15115, 1, 0.3333333333333333), (12893, 1, 0.1), (13024, 1, 0.3333333333333333), (15408, 1, 0.07142857142857142), (15872, 1, 0.1111111111111111), (12170, 1, 0.5), (11820, 1, 0.14285714285714285), (17168, 1, 0.3333333333333333), (13578, 1, 0.25), (16018, 1, 0.1111111111111111), (15628, 1, 0.3333333333333333), (14201, 1, 0.25), (15861, 1, 0.2), (15652, 1, 0.25), (16158, 1, 0.07692307692307693), (13146, 1, 0.09090909090909091), (12880, 1, 0.09090909090909091), (12958, 1, 0.2), (12077, 1, 0.1111111111111111), (15662, 1, 0.3333333333333333), (16796, 1, 0.5), (14230, 1, 0.14285714285714285), (13620, 1, 0.2), (12009, 1, 0.3333333333333333), (12512, 1, 0.25), (15870, 1, 0.08333333333333333), (17362, 1, 0.2), (12516, 1, 0.2), (16688, 1, 0.2), (13092, 1, 0.3333333333333333), (14404, 1, 0.25), (15633, 1, 0.25), (13090, 1, 0.14285714285714285), (15024, 1, 0.3333333333333333), (12481, 1, 0.3333333333333333), (14977, 1, 0.14285714285714285

In [0]:
# (word, [(did,freq,tfidf), ...])
# We easily obtain idf as 1/len(postinglist_tf)
# idf = 1/len(postinglist_tf)


num_docs = len(input2.collect())
print("Number of documents = ", num_docs)

word_postinglist_freq_tfidf = word_postinglist_freq_tf.mapValues(lambda postinglist_tf: [(x[0],x[1],x[2]/len(postinglist_tf)) for x in postinglist_tf])

print(word_postinglist_freq_tfidf.take(4))


Number of documents =  19026
[('apr', [(13972, 1, 5.269814502529511e-05), (17227, 1, 3.5132096683530073e-05), (15115, 1, 7.026419336706015e-05), (12893, 1, 2.1079258010118044e-05), (13024, 1, 7.026419336706015e-05), (15408, 1, 1.505661286437003e-05), (15872, 1, 2.3421397789020048e-05), (12170, 1, 0.00010539629005059021), (11820, 1, 3.011322572874006e-05), (17168, 1, 7.026419336706015e-05), (13578, 1, 5.269814502529511e-05), (16018, 1, 2.3421397789020048e-05), (15628, 1, 7.026419336706015e-05), (14201, 1, 5.269814502529511e-05), (15861, 1, 4.215851602023609e-05), (15652, 1, 5.269814502529511e-05), (16158, 1, 1.621481385393696e-05), (13146, 1, 1.916296182738004e-05), (12880, 1, 1.916296182738004e-05), (12958, 1, 4.215851602023609e-05), (12077, 1, 2.3421397789020048e-05), (15662, 1, 7.026419336706015e-05), (16796, 1, 0.00010539629005059021), (14230, 1, 3.011322572874006e-05), (13620, 1, 4.215851602023609e-05), (12009, 1, 7.026419336706015e-05), (12512, 1, 5.269814502529511e-05), (15870, 1

In [0]:
# Now, we would like to obtain the magnitude of each doc.
# First, produce (did, (freq,tfidf)) for each word of doc did; 
# We don't need the word itself, just its (freq,tfidf). 
# Then, do reduceByKey on these tuples and obtain maxfreq and 
# magnitude (squared) for each document. 

# # RDD of (did,(freq,tfidf)) tuples
did_freq_tfidfsq_rdd = word_postinglist_freq_tfidf.flatMap(lambda x: x[1]).map(lambda x: (x[0], (x[1],x[2]**2)))

doc_maxf_mag = did_freq_tfidfsq_rdd.reduceByKey(lambda x1,x2: (max(x1[0], x2[0]), x1[1] + x2[1]))
print(doc_maxf_mag.take(5))

[(13972, (4, 0.008940793196793672)), (13024, (3, 0.0008485701913941564)), (15408, (14, 0.015251842053296681)), (15872, (9, 0.012389382565163249)), (12170, (2, 0.35032180909841704))]


In [0]:
!rm -rf inv_idx
word_postinglist_freq_tfidf.saveAsTextFile("inv_idx");

In [0]:
!rm -rf doc_mag
doc_maxf_mag.saveAsTextFile("doc_mag");

In [0]:
!ls

doc_mag		inv_idx			   spark-2.4.4-bin-hadoop2.7.tgz
input_docs	sample_data
input_docs.zip	spark-2.4.4-bin-hadoop2.7


In [0]:
!ls -lrt inv_idx
!head inv_idx/part-00001
!wc -l inv_idx/part-00000
!wc -l inv_idx/part-00001
!cat inv_idx/part-00000 inv_idx/part-00001 > inv_idx.txt
!wc -l inv_idx.txt
files.download('inv_idx.txt')

total 37772
-rw-r--r-- 1 root root 20689591 Apr  2 02:03 part-00000
-rw-r--r-- 1 root root 17983609 Apr  2 02:03 part-00001
-rw-r--r-- 1 root root        0 Apr  2 02:03 _SUCCESS
('ia', [(13972, 1, 0.08333333333333333), (20093, 3, 0.19999999999999998), (13805, 1, 0.16666666666666666)])
('direct', [(13972, 1, 0.0012755102040816326), (1312, 1, 0.0002834467120181406), (20868, 1, 0.00046382189239332097), (7356, 1, 0.0002834467120181406), (14492, 1, 0.00042517006802721087), (20437, 1, 0.0006377551020408163), (137, 2, 0.0012755102040816326), (16863, 1, 0.0017006802721088435), (4422, 1, 0.00036443148688046647), (15781, 1, 0.0005102040816326531), (3287, 3, 0.0011773940345368918), (10873, 1, 0.0012755102040816326), (12072, 1, 0.0008503401360544217), (4256, 1, 0.0008503401360544217), (20750, 1, 0.0003924646781789639), (8772, 1, 0.0010204081632653062), (12477, 1, 0.00031887755102040814), (6238, 1, 0.0005668934240362812), (16900, 1, 0.0010204081632653062), (12417, 1, 0.0008503401360544217), (12052,

In [0]:
!ls -lrt doc_mag
!head doc_mag/part-00000
!head doc_mag/part-00001
!wc -l doc_mag/part-00000
!wc -l doc_mag/part-00001

!cat doc_mag/part-00000 doc_mag/part-00001 > doc_mag.txt
!wc -l doc_mag.txt
files.download('doc_mag.txt')

total 636
-rw-r--r-- 1 root root 324193 Apr  2 02:03 part-00001
-rw-r--r-- 1 root root 323218 Apr  2 02:03 part-00000
-rw-r--r-- 1 root root      0 Apr  2 02:03 _SUCCESS
(13972, (4, 0.008940793196793672))
(13024, (3, 0.0008485701913941564))
(15408, (14, 0.015251842053296681))
(15872, (9, 0.012389382565163249))
(12170, (2, 0.35032180909841704))
(11820, (7, 0.0034631207486893615))
(17168, (3, 0.1876581139358987))
(13578, (4, 0.006341952093488519))
(16018, (9, 0.05963037402807093))
(15628, (3, 0.0009311980551929494))
(17227, (6, 0.0021756257766608065))
(15115, (3, 0.11313174161093323))
(12893, (10, 0.0011115026270279546))
(14201, (4, 0.004443998449495605))
(15861, (5, 0.0960903436298313))
(12077, (9, 0.0013811313160985612))
(12009, (3, 0.06334766035614854))
(15633, (4, 0.03477801151549394))
(12481, (3, 0.053625227668946084))
(14977, (7, 0.03855669126866296))
9500 doc_mag/part-00000
9526 doc_mag/part-00001
19026 doc_mag.txt
